In [277]:
%load_ext autoreload
%autoreload 2

In [278]:
import os
import sys
sys.path.append(os.path.realpath('..'))

In [280]:
import numpy as np

from math import sqrt
from sympy import Symbol, symbols, simplify, collect, rsolve, Function

from tracking.kalman.convergence import kalman_gain_pv_numeric

In [9]:
Kx, Kv, t, a = symbols("K_v, K_x, t, a")

In [48]:
def x(i):
    return a * t**2 * (i**2 / 2)

Rs = symbols(",".join([f"R_{i+1}" for i in range(10)]))
def R(i):
    return Rs[i-1]

In [49]:
x0, v0 = 0, 0

In [50]:
x1 = x0 + v0*t + Kx*(x(1) + R(1) - v0*t)
v1 = v0 + Kv*(x(1) + R(1) - v0*t)

display(x1, v1)

K_v*(R_1 + 0.5*a*t**2)

K_x*(R_1 + 0.5*a*t**2)

In [57]:
def x_hat(i):
    if i == 0:
        return 0
    eq = (1-Kx)*(x_hat(i-1) + t*v_hat(i-1)) + Kx*(x(i) + R(i))
    return collect(eq.expand(), Rs[:i])

def v_hat(i):
    if i == 0:
        return 0
    eq = v_hat(i-1) + Kv*(x(i)+R(i) - x_hat(i-1) - t*v_hat(i-1))
    return collect(eq.expand(), Rs[:i])

In [58]:
x_hat(1)

K_v*R_1 + 0.5*K_v*a*t**2

In [67]:
x_hat(2)

-0.5*K_v**2*a*t**2 - 0.5*K_v*K_x*a*t**3 + K_v*R_2 + 2.5*K_v*a*t**2 + 0.5*K_x*a*t**3 + R_1*(-K_v**2 - K_v*K_x*t + K_v + K_x*t)

In [68]:
x_hat(3)

0.5*K_v**3*a*t**2 + 1.0*K_v**2*K_x*a*t**3 - 3.0*K_v**2*a*t**2 + 0.5*K_v*K_x**2*a*t**4 - 4.0*K_v*K_x*a*t**3 + K_v*R_3 + 7.0*K_v*a*t**2 - 0.5*K_x**2*a*t**4 + 3.0*K_x*a*t**3 + R_1*(K_v**3 + 2*K_v**2*K_x*t - 2*K_v**2 + K_v*K_x**2*t**2 - 4*K_v*K_x*t + K_v - K_x**2*t**2 + 2*K_x*t) + R_2*(-K_v**2 - K_v*K_x*t + K_v + K_x*t)

In [69]:
x_hat(4)

-0.5*K_v**4*a*t**2 - 1.5*K_v**3*K_x*a*t**3 + 3.5*K_v**3*a*t**2 - 1.5*K_v**2*K_x**2*a*t**4 + 8.5*K_v**2*K_x*a*t**3 - 10.0*K_v**2*a*t**2 - 0.5*K_v*K_x**3*a*t**5 + 5.5*K_v*K_x**2*a*t**4 - 17.0*K_v*K_x*a*t**3 + K_v*R_4 + 15.0*K_v*a*t**2 + 0.5*K_x**3*a*t**5 - 4.0*K_x**2*a*t**4 + 10.0*K_x*a*t**3 + R_1*(-K_v**4 - 3*K_v**3*K_x*t + 3*K_v**3 - 3*K_v**2*K_x**2*t**2 + 9*K_v**2*K_x*t - 3*K_v**2 - K_v*K_x**3*t**3 + 7*K_v*K_x**2*t**2 - 9*K_v*K_x*t + K_v + K_x**3*t**3 - 4*K_x**2*t**2 + 3*K_x*t) + R_2*(K_v**3 + 2*K_v**2*K_x*t - 2*K_v**2 + K_v*K_x**2*t**2 - 4*K_v*K_x*t + K_v - K_x**2*t**2 + 2*K_x*t) + R_3*(-K_v**2 - K_v*K_x*t + K_v + K_x*t)

In [139]:
#x_hat = Function("x")
#v_hat = Function("v")

In [75]:
def var_x(i):
    if i == 1:
        return Kx**2
    return (1-Kx)**2 * var_x(i-1) + ((1-Kx)*t)**2 * var_v(i-1) + Kx**2

def var_v(i):
    if i == 1:
        return Kv**2
    return (1-t*Kv)**2 * var_v(i-1) + Kv**2 * var_x(i-1) + Kv**2

In [138]:
collect(var_x(3).expand(), [Kx, Kv])

K_v**6 - 4*K_v**5 + K_v**4*(2*K_x**2*t**2 + 7) + K_v**3*(-6*K_x**2*t**2 - 6) + K_v**2*(K_x**4*t**4 - 2*K_x**3*t**3 + 9*K_x**2*t**2 + 3) + K_v*(-2*K_x**4*t**4 + 4*K_x**3*t**3 - 8*K_x**2*t**2) + K_x**4*t**4 - 2*K_x**3*t**3 + 3*K_x**2*t**2

In [310]:
Q = np.array([[1.0/3, .5], [.5, 1]]) * .1**2
kalman_gain_pv_numeric(.7569, Q, 1, n=100).squeeze()

array([0.38088302, 0.09044136])

In [313]:
def converge_variances_numerically(Kx, Kv, R=1, n=100):
    Q = np.array([[1.0/3, .5], [.5, 1]]) * .1**2
    Kx, Kv = kalman_gain_pv_numeric(R, Q, 1).squeeze()
    Vx, Vv, Cov = Kx*Kx * R, Kv*Kv * R, Kx*Kv*R
    
    for i in range(n):
        Kx, Kv = kalman_gain_pv_numeric(R, Q, n=i+2).squeeze()
        Kx2 = Kx*Kx
        dKx = (1-Kx)
        dKx2 = dKx*dKx
        Kv2 = Kv*Kv
        dKv = (1-Kv)
        dKv2 = dKv*dKv
        KxKv = Kx*Kv
        
        nVx = dKx2 * (Vx + Vv + 2*Cov) + Kx2 * R
        
        nVv = dKv2 * Vv + \
              Kv2 * Vx \
              - 2*dKv*Kv*Cov + \
              Kv2 * R
        
        Cov = Cov*dKx*(1-2*Kv) + \
              dKx*dKv*Vv \
              - dKx*Kv*Vx + \
              KxKv*R
                
        Vx = nVx
        Vv = nVv
        
    return Vx, Vv, Cov

In [314]:
Vx, Vv, Cov = converge_variances_numerically(0.380883, 0.090441, R=0.7569, n=200)
Vx, Vv, Cov

(0.23212141364258104, 0.010327732695793287, 0.0383301482976182)

In [315]:
Vx+Vv+2*Cov, Vv

(0.31910944293361077, 0.010327732695793287)